In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
            # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from tqdm import tqdm
import numpy as np
import pandas as pd
import zipfile
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import tensorflow as tf
import shutil

In [ ]:
df_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
from pandas_profiling import ProfileReport
train_profile = ProfileReport(df_train, title="Train Data")
test_profile  = ProfileReport(df_test, title="Test Data")
display(train_profile)

In [ ]:
display(test_profile)

In [ ]:
X = df_train.drop(['Id'],axis=1)
X_test = df_test.drop(['Id'],axis=1)
X['Pawpularity'] = X['Pawpularity']/100
y = X['Pawpularity']
X = X.drop('Pawpularity',axis=1)

In [ ]:
model = XGBRegressor()
model.fit (X,y)
feature_picked = []
importance = model.feature_importances_
for i,v in enumerate(importance):
    print("feature : {}, score : %.5f" .format(X.columns[i]) %(v))
    if v > 0.08 :
        feature_picked.append(X.columns[i])
        print('input {}'.format(X.columns[i]))
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
X['classification'] = X[feature_picked].apply(lambda row : ''.join(row.values.astype(str)),axis=1)
X_test['classification'] = X_test[feature_picked].apply(lambda row : ''.join(row.values.astype(str)),axis=1)

In [ ]:
groups = np.array(X['classification']).copy().tolist()

In [ ]:
X['fpath'] = df_train['Id'].apply(lambda x : '../input/petfinder-pawpularity-score/train/' + x + '.jpg')
X_test['fpath'] = df_test['Id'].apply(lambda x : '../input/petfinder-pawpularity-score/test/' + x + '.jpg')
meta_X = X[['classification','fpath']].copy()
meta_X['Pawpularity'] = y.copy()

In [ ]:
result = []
groupk = GroupKFold(n_splits=5)
for train_index, test_index in groupk.split(meta_X, y,groups=groups):
    X_train = meta_X.iloc[train_index]
    X_valid = meta_X.iloc[test_index]
    result.append((X_train, X_valid))

train_fold_1, valid_fold_1 = result[0][0], result[0][1]
train_fold_2, valid_fold_2 = result[1][0], result[1][1]
train_fold_3, valid_fold_3 = result[2][0], result[2][1]
train_fold_4, valid_fold_4 = result[3][0], result[3][1]
train_fold_5, valid_fold_5 = result[4][0], result[4][1]

In [ ]:
valid_fold_3.groupby('classification').get_group('000000').count()

In [ ]:
valid_fold_3.head()

In [ ]:
X_test.head()
test_fold = X_test[['classification','fpath']].copy()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_fold_1,
    x_col = 'fpath',
    y_col = 'Pawpularity',
    target_size=(224,224),
    batch_size=3,
    class_mode='raw'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_fold_1,
    x_col = 'fpath',
    y_col = 'Pawpularity',
    target_size=(224,224),
    batch_size=3,
    class_mode='raw'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_fold,
    x_col = 'fpath',
    target_size=(224,224),
    batch_size=1,
    class_mode=None
)

In [ ]:
def get_model():
    base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights=None, input_shape=(224,224,3))

    out = tf.keras.layers.Flatten()(base_model.output)
    out = tf.keras.layers.Dense(2048, activation='relu')(out)
    out = tf.keras.layers.Dropout(0.5)(out)
    out = tf.keras.layers.Dense(2048, activation='relu')(out)
    out = tf.keras.layers.Dropout(0.5)(out)
    output = tf.keras.layers.Dense(10, activation='softmax')(out)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    loss_fn = tf.keras.losses.MeanSquaredError()
    model.compile(loss=loss_fn,optimizer=opt,metrics=['mae', 'mse'])

    return model

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
model = get_model()

for i in range(len(result)):
    j=0
    train_generator = datagen.flow_from_dataframe(
    result[i][j],
    x_col = 'fpath',
    y_col = 'Pawpularity',
    target_size=(224,224),
    batch_size=3,
    class_mode='raw'
    )

    valid_generator = datagen.flow_from_dataframe(
    result[i][j+1],
    x_col = 'fpath',
    y_col = 'Pawpularity',
    target_size=(224,224),
    batch_size=3,
    class_mode='raw'
    )

    model.fit(
        train_generator,
        epochs=1,
        validation_data=valid_generator,
        shuffle=True,
        verbose=1
    )

In [ ]:
test_generator = datagen.flow_from_dataframe(
test_fold,
x_col = 'fpath',
target_size=(224,224),
batch_size=3,
class_mode=None
)
results = model.predict(test_generator)

In [ ]:
df_test.head()

In [ ]:
results

In [ ]:
df_test.loc[1][0]

In [ ]:
submission = []

for index, result in enumerate(results):
    print(index, result[0])
    submission.append([df_test.loc[index][0], float(result[0])])
    
    
df = pd.DataFrame(submission, columns=['Id', 'Pawpularity'])
df = df.sort_values('Id')
df.to_csv('submission.csv', index=False)
df

In [ ]:
df.head()